# UHFF Android Builder - Fixed Version 🔧

This notebook addresses common buildozer and python-for-android toolchain errors when building Android APKs in Google Colab.

## ⚡ Quick Fix for Your Error

The error you encountered is related to python-for-android toolchain configuration and architecture compatibility. This notebook provides step-by-step solutions.

## 🎯 What We'll Fix

- ✅ Python-for-android toolchain setup
- ✅ Architecture build issues (arm64-v8a, armeabi-v7a)  
- ✅ NDK/SDK version compatibility
- ✅ Pygame and NumPy compilation problems
- ✅ Buildozer configuration optimization

---

## 🚀 Section 1: Install and Setup Build Environment

Let's start by setting up a clean, compatible build environment that addresses the common issues.

In [ ]:
# 🔧 STEP 1: Clean Environment Setup
print("🧹 Setting up clean build environment...")

# Remove any existing problematic installations
!pip uninstall -y buildozer python-for-android

# Update system packages with specific versions that work well together
!apt-get update -qq
!apt-get install -y -qq \
    openjdk-8-jdk \
    wget unzip git \
    build-essential \
    libffi-dev \
    libssl-dev \
    python3-dev \
    zlib1g-dev \
    libjpeg-dev \
    cmake

# Install specific compatible versions
!pip install --upgrade pip setuptools wheel
!pip install 'buildozer==1.5.0'
!pip install 'python-for-android==2024.1.21'
!pip install 'cython<3.0'

print("✅ Environment setup complete!")

# Verify installations
import subprocess
import sys

def check_version(command, name):
    try:
        result = subprocess.run(command.split(), capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ {name}: {result.stdout.strip()}")
        else:
            print(f"❌ {name}: Not found or error")
    except:
        print(f"❌ {name}: Not installed")

check_version("buildozer version", "Buildozer")
check_version("python3 --version", "Python")
check_version("java -version", "Java")

In [ ]:
# 🔧 STEP 2: Setup Android SDK with Compatible Versions
import os
import zipfile
from pathlib import Path

# Set up Android SDK with versions that work well with current p4a
android_home = Path.home() / "android-sdk"
android_home.mkdir(exist_ok=True)

# Use older, more stable Android SDK tools
if not (android_home / "cmdline-tools" / "latest").exists():
    print("📥 Downloading compatible Android SDK...")
    
    # Use older cmdline-tools version that works better
    !wget -q https://dl.google.com/android/repository/commandlinetools-linux-9477386_latest.zip -O cmdtools.zip
    
    with zipfile.ZipFile('cmdtools.zip', 'r') as zip_ref:
        zip_ref.extractall(android_home / "cmdline-tools")
    
    import shutil
    shutil.move(
        str(android_home / "cmdline-tools" / "cmdline-tools"),
        str(android_home / "cmdline-tools" / "latest")
    )
    os.remove('cmdtools.zip')
    print("✅ Android SDK tools installed")

# Set critical environment variables
os.environ['ANDROID_HOME'] = str(android_home)
os.environ['ANDROID_SDK_ROOT'] = str(android_home)
os.environ['PATH'] = f"{android_home}/cmdline-tools/latest/bin:{android_home}/platform-tools:{os.environ['PATH']}"

# Use Java 8 (more compatible with older Android tools)
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

# Install specific Android SDK components that work well
if not (android_home / "platform-tools").exists():
    print("📦 Installing Android SDK components...")
    
    # Accept licenses first
    !yes | {android_home}/cmdline-tools/latest/bin/sdkmanager --licenses
    
    # Install compatible versions
    !{android_home}/cmdline-tools/latest/bin/sdkmanager "platform-tools" "platforms;android-30" "build-tools;30.0.3"
    
    print("✅ Android SDK ready!")

print(f"Android Home: {os.environ.get('ANDROID_HOME')}")
print(f"Java Home: {os.environ.get('JAVA_HOME')}")

## 🔍 Section 2: Diagnose Common Build Issues

Let's analyze your specific error and set up diagnostic tools.

In [ ]:
# 🔍 STEP 3: Error Diagnosis and Environment Check
def diagnose_build_environment():
    """Diagnose common build issues and environment problems"""
    
    print("🔍 DIAGNOSING BUILD ENVIRONMENT")
    print("=" * 50)
    
    issues_found = []
    
    # Check Python version compatibility
    import sys
    python_version = sys.version_info
    print(f"Python Version: {python_version.major}.{python_version.minor}.{python_version.micro}")
    
    if python_version >= (3, 12):
        issues_found.append("Python 3.12+ may have compatibility issues with older p4a")
        print("⚠️  Python 3.12+ detected - may need compatibility fixes")
    else:
        print("✅ Python version compatible")
    
    # Check Java version
    import subprocess
    try:
        java_result = subprocess.run(['java', '-version'], capture_output=True, text=True)
        if 'openjdk version "1.8' in java_result.stderr:
            print("✅ Java 8 detected (recommended)")
        elif 'openjdk version "11' in java_result.stderr:
            print("⚠️  Java 11 detected (may work, Java 8 preferred)")
        else:
            issues_found.append("Java version may be incompatible")
            print("❌ Java version issue detected")
    except:
        issues_found.append("Java not found or not accessible")
        print("❌ Java not found")
    
    # Check environment variables
    required_vars = ['ANDROID_HOME', 'JAVA_HOME']
    for var in required_vars:
        if var in os.environ:
            print(f"✅ {var}: {os.environ[var]}")
        else:
            issues_found.append(f"Missing {var} environment variable")
            print(f"❌ {var}: Not set")
    
    # Check buildozer version
    try:
        result = subprocess.run(['buildozer', 'version'], capture_output=True, text=True)
        print(f"✅ Buildozer: {result.stdout.strip()}")
    except:
        issues_found.append("Buildozer not accessible")
        print("❌ Buildozer: Not found")
    
    print(f"\n📊 Issues Found: {len(issues_found)}")
    for i, issue in enumerate(issues_found, 1):
        print(f"  {i}. {issue}")
    
    return issues_found

# Run diagnosis
issues = diagnose_build_environment()

# Analyze the specific error from your build
print("\n🎯 ANALYZING YOUR SPECIFIC ERROR")
print("=" * 50)
print("Your error shows:")
print("1. ❌ Python-for-android toolchain creation failed")
print("2. ❌ Multi-architecture build (arm64-v8a + armeabi-v7a) issues")
print("3. ❌ NDK API level 21 configuration problems")
print("4. ❌ Pygame/NumPy compilation conflicts")

print("\n💡 RECOMMENDED FIXES:")
print("1. 🔧 Use single architecture build first")
print("2. 🔧 Update buildozer.spec with compatible settings")
print("3. 🔧 Use older, stable NDK version")
print("4. 🔧 Simplify requirements list")

## ⚙️ Section 3: Configure Buildozer Spec File

Let's create an optimized buildozer.spec that addresses the common build failures.

In [ ]:
# ⚙️ STEP 4: Create Fixed buildozer.spec Configuration

# Create project directory
project_dir = Path("/content/uhff-project")
project_dir.mkdir(exist_ok=True)
os.chdir(project_dir)

# Create optimized buildozer.spec that fixes common issues
buildozer_spec_fixed = """[app]
title = UHFF Visualization App
package.name = uhffvis
package.domain = com.uhff

source.dir = .
source.include_exts = py,png,jpg,kv,atlas,json
version = 1.0

# CRITICAL: Simplified requirements to avoid conflicts
requirements = python3,kivy,pygame,numpy

# Single architecture first (easier to debug)
# We'll add multiple archs after this works
orientation = landscape

[buildozer]
log_level = 2

# Android specific - FIXED CONFIGURATION
android.api = 30
android.minapi = 21

# Use older, more stable NDK
android.ndk = 21b
android.ndk_api = 21

# SINGLE ARCHITECTURE FIRST - this is key!
android.archs = arm64-v8a

# Critical settings for pygame
android.accept_sdk_license = True
android.skip_update = True

# Use SDL2 bootstrap (required for pygame)
p4a.bootstrap = sdl2

# Specific p4a branch that works well with pygame
p4a.branch = develop

# Add some extra build options to help with compilation
p4a.extra_args = --arch=arm64-v8a

# Permissions that pygame might need
android.permissions = INTERNET,WRITE_EXTERNAL_STORAGE

[app:source.exclude_patterns]
# Exclude unnecessary files
*.pyc
.git/*
__pycache__/*
*.log
"""

# Write the fixed buildozer.spec
with open("buildozer.spec", "w") as f:
    f.write(buildozer_spec_fixed)

print("✅ Created fixed buildozer.spec with:")
print("  🔧 Single architecture (arm64-v8a)")
print("  🔧 Stable Android API 30")
print("  🔧 Older NDK (21b) for better compatibility")
print("  🔧 Simplified requirements")
print("  🔧 SDL2 bootstrap for pygame")
print("  🔧 P4A develop branch")

# Show the configuration
print("\n📋 Buildozer Configuration Summary:")
!cat buildozer.spec | grep -E "^(title|requirements|android\.api|android\.ndk|android\.archs|p4a\.bootstrap)" | head -10

## 📁 Section 4: Upload Your Project Files

Now let's get your UHFF project files uploaded and properly configured.

In [ ]:
# 📁 STEP 5: Upload and Configure Project Files
from google.colab import files
import zipfile
import shutil

print("📂 Upload your UHFF project files")
print("Choose one option:")
print("1. Upload individual Python files (main.py, uhff_visualization.py)")
print("2. Upload entire project as ZIP file")
print("3. Clone from GitHub repository")

choice = input("Enter your choice (1, 2, or 3): ")

if choice == "1":
    print("\n📤 Please upload your Python files:")
    print("Required files: main.py, uhff_visualization.py")
    print("Optional: any other .py files")
    
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"✅ Uploaded: {filename}")

elif choice == "2":
    print("\n📤 Please upload your project ZIP file:")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('.')
            os.remove(filename)
            print(f"✅ Extracted: {filename}")

elif choice == "3":
    repo_url = input("Enter GitHub repository URL: ")
    if repo_url:
        !git clone {repo_url} temp_repo
        # Move files from temp_repo to current directory
        !cp -r temp_repo/* .
        !rm -rf temp_repo
        print("✅ Repository cloned")

# Verify we have the main files
required_files = ['main.py']
missing_files = []

for file in required_files:
    if not Path(file).exists():
        missing_files.append(file)

if missing_files:
    print(f"\n⚠️ Missing required files: {missing_files}")
    print("Creating placeholder files...")
    
    if 'main.py' in missing_files:
        placeholder_main = '''
import pygame
import numpy as np

# Placeholder main.py - replace with your actual code
def main():
    print("UHFF Visualization - Replace this with your actual main.py")

if __name__ == "__main__":
    main()
'''
        with open('main.py', 'w') as f:
            f.write(placeholder_main)
        print("✅ Created placeholder main.py")

# Show project structure
print(f"\n📋 Project files:")
!ls -la *.py 2>/dev/null || echo "No Python files found"

# Verify file contents (first few lines)
if Path('main.py').exists():
    print(f"\n🔍 main.py preview (first 10 lines):")
    !head -10 main.py

## 🛠️ Section 5: Debug Python-for-Android Errors

Let's address the specific p4a toolchain issues and test our fixes incrementally.

In [ ]:
# 🛠️ STEP 6: Fix P4A Issues and Test Build
print("🔧 Fixing Python-for-Android toolchain issues...")

# Clear any existing buildozer cache that might cause issues
print("🧹 Cleaning previous build artifacts...")
!rm -rf .buildozer 2>/dev/null || true
!buildozer android clean 2>/dev/null || true

# Initialize buildozer in diagnostic mode
print("\n🔍 Testing buildozer initialization...")
result = !buildozer android debug --verbose 2>&1 | head -50

# Check for common error patterns
error_patterns = [
    "No such file or directory",
    "Permission denied", 
    "NDK not found",
    "SDK not found",
    "Java not found",
    "recipe.*failed",
    "compilation terminated"
]

found_errors = []
for line in result:
    for pattern in error_patterns:
        if pattern.lower() in line.lower():
            found_errors.append(line.strip())
            break

if found_errors:
    print("\n⚠️ Issues detected in initialization:")
    for error in found_errors[:5]:  # Show first 5 errors
        print(f"  📍 {error}")
else:
    print("✅ No obvious errors in initialization")

# Test p4a directly to diagnose toolchain issues
print("\n🔬 Testing python-for-android directly...")
!python3 -m pythonforandroid.toolchain recipes | head -10

print("\n🔬 Testing specific recipes we need...")
recipes_to_check = ['pygame', 'numpy', 'python3']
for recipe in recipes_to_check:
    result = !python3 -m pythonforandroid.toolchain recipes | grep -i {recipe}
    if result:
        print(f"✅ {recipe}: Available")
    else:
        print(f"⚠️ {recipe}: May have issues")

# Create a minimal test to verify our setup works
print("\n🧪 Creating minimal test build...")
minimal_spec = """[app]
title = Test App
package.name = testapp
package.domain = com.test
source.dir = .
version = 0.1
requirements = python3

[buildozer]
log_level = 2
android.api = 30
android.minapi = 21
android.ndk = 21b
android.archs = arm64-v8a
android.accept_sdk_license = True
p4a.bootstrap = sdl2
"""

# Save current buildozer.spec and create test version
!cp buildozer.spec buildozer.spec.backup
with open("buildozer.spec", "w") as f:
    f.write(minimal_spec)

print("✅ Created minimal test configuration")
print("🎯 Ready for test build in next cell")

## ✅ Section 6: Test Build Configuration

Let's run a minimal test build to verify our fixes work, then gradually add complexity.

In [ ]:
# ✅ STEP 7: Minimal Test Build
print("🧪 Running minimal test build to verify setup...")
print("⏱️ This should take 5-10 minutes (much faster than full build)")

# Create minimal main.py for testing
test_main = '''
print("Minimal Android app test")

def main():
    print("UHFF Test App - Android build successful!")

if __name__ == "__main__":
    main()
'''

with open("main.py", "w") as f:
    f.write(test_main)

# Run the minimal build
import time
start_time = time.time()

print("🚀 Starting minimal build...")
result = !buildozer android debug 2>&1

# Check if build succeeded
if "BUILD SUCCESSFUL" in ' '.join(result) or any("*.apk" in line for line in result):
    elapsed = time.time() - start_time
    print(f"✅ MINIMAL BUILD SUCCESSFUL! ({elapsed:.1f} seconds)")
    print("🎉 Environment is working correctly!")
    
    # Show APK if created
    !ls -la bin/*.apk 2>/dev/null || echo "APK file checking..."
    
elif "BUILD FAILED" in ' '.join(result):
    print("❌ Minimal build failed. Let's diagnose...")
    
    # Show last 20 lines of output for errors
    print("\n🔍 Last 20 lines of build output:")
    for line in result[-20:]:
        print(line)
        
    # Check for specific error patterns
    common_errors = {
        "NDK": "Android NDK issue - check NDK installation",
        "SDK": "Android SDK issue - check SDK components", 
        "Java": "Java version issue - need Java 8",
        "recipe": "Recipe compilation issue - check requirements",
        "permission": "Permission issue - check file permissions"
    }
    
    print("\n🎯 Error analysis:")
    for error_type, description in common_errors.items():
        if any(error_type.lower() in line.lower() for line in result):
            print(f"  📍 {error_type}: {description}")

else:
    print("⏳ Build in progress or incomplete output...")
    print("Last few lines:")
    for line in result[-10:]:
        print(line)

print(f"\n📊 Build attempt completed")

In [ ]:
# 🎯 STEP 8: Build UHFF App (If Minimal Build Succeeded)
print("🚀 Building full UHFF application...")

# Restore the full buildozer.spec for UHFF
uhff_buildozer_spec = """[app]
title = UHFF Visualization App
package.name = uhffvis
package.domain = com.uhff

source.dir = .
source.include_exts = py,png,jpg,kv,atlas,json
version = 1.0

# Start with pygame only, add numpy later if this works
requirements = python3,pygame

# Single architecture for now
orientation = landscape

[buildozer]
log_level = 2

android.api = 30
android.minapi = 21
android.ndk = 21b
android.ndk_api = 21
android.archs = arm64-v8a

android.accept_sdk_license = True
android.skip_update = True

# SDL2 bootstrap required for pygame
p4a.bootstrap = sdl2
p4a.branch = develop

# Extra arguments to help with pygame compilation
p4a.extra_args = --arch=arm64-v8a

# Basic permissions
android.permissions = INTERNET
"""

with open("buildozer.spec", "w") as f:
    f.write(uhff_buildozer_spec)

print("✅ Created UHFF buildozer.spec")

# Only proceed if user confirms minimal build worked
proceed = input("\\nDid the minimal build succeed? (y/n): ")

if proceed.lower() == 'y':
    print("🎯 Building UHFF app with pygame...")
    
    # Clean previous build
    !buildozer android clean
    
    # Start build
    !buildozer android debug
    
    # Check results
    if Path("bin").exists():
        apk_files = list(Path("bin").glob("*.apk"))
        if apk_files:
            for apk in apk_files:
                size_mb = apk.stat().st_size / 1024 / 1024
                print(f"🎉 SUCCESS! Generated: {apk.name} ({size_mb:.1f} MB)")
        else:
            print("❌ No APK files found in bin directory")
    else:
        print("❌ No bin directory found - build likely failed")
        
else:
    print("⚠️ Fix minimal build issues first, then run this cell again")
    print("💡 Common fixes:")
    print("  1. Check Java version (should be Java 8)")
    print("  2. Verify Android SDK/NDK paths")
    print("  3. Clear buildozer cache: !rm -rf .buildozer")
    print("  4. Try different NDK version in buildozer.spec")

## 📥 Section 7: Download APK and Troubleshooting

Final steps to get your APK and troubleshoot any remaining issues.

In [ ]:
# 📥 STEP 9: Download APK and Final Steps
from google.colab import files

print("📱 Checking for generated APK files...")

bin_dir = Path("bin")
if bin_dir.exists():
    apk_files = list(bin_dir.glob("*.apk"))
    
    if apk_files:
        print(f"🎉 Found {len(apk_files)} APK file(s):")
        
        for apk_file in apk_files:
            size_mb = apk_file.stat().st_size / 1024 / 1024
            print(f"  📱 {apk_file.name} ({size_mb:.1f} MB)")
            
            # Download the APK
            print(f"📥 Downloading {apk_file.name}...")
            files.download(str(apk_file))
        
        print("\\n✅ SUCCESS! Your UHFF Android app is ready!")
        print("\\n📱 Installation Instructions:")
        print("1. Transfer the APK to your Android device")
        print("2. Enable 'Unknown Sources' in Android settings")
        print("3. Tap the APK file to install")
        print("4. Launch 'UHFF Visualization App'")
        
        print("\\n🎮 App Features:")
        print("✅ Touch controls for Android")
        print("✅ Virtual on-screen buttons") 
        print("✅ 11 mathematical visualization scenes")
        print("✅ Parameter editing with sliders")
        print("✅ Save/load presets")
        
    else:
        print("❌ No APK files found in bin directory")
        print("\\n🔍 Let's check what happened...")
        
        # Show bin directory contents
        print("Bin directory contents:")
        !ls -la bin/ 2>/dev/null || echo "Bin directory is empty or doesn't exist"
        
        # Check for build logs
        if Path(".buildozer").exists():
            print("\\n📋 Checking build logs...")
            !find .buildozer -name "*.log" -exec echo "=== {} ===" \\; -exec tail -10 {} \\; 2>/dev/null | head -50
            
else:
    print("❌ No bin directory found - build did not complete successfully")
    
    print("\\n🛠️ TROUBLESHOOTING STEPS:")
    print("1. 🔧 Check build errors above")
    print("2. 🔧 Try different NDK version:")
    print("   - Change 'android.ndk = 21b' to 'android.ndk = 23b'")
    print("3. 🔧 Try single requirement:")
    print("   - Change 'requirements = python3,pygame' to 'requirements = python3'")
    print("4. 🔧 Clean and retry:")
    print("   - Run: !rm -rf .buildozer")
    print("   - Then re-run build")

# Show final project structure
print("\\n📂 Final project structure:")
!ls -la *.py *.spec 2>/dev/null || echo "No project files found"

## 🔧 Emergency Fixes & Alternative Approaches

If the above didn't work, here are additional troubleshooting steps and alternative approaches.

In [ ]:
# 🚑 EMERGENCY FIXES - Run if build still fails

print("🚑 EMERGENCY TROUBLESHOOTING")
print("="*50)

# Fix 1: Try with Kivy instead of direct pygame
print("\\n🔧 FIX 1: Kivy-based approach (more reliable)")
kivy_spec = """[app]
title = UHFF Viz
package.name = uhffviz  
package.domain = com.uhff
source.dir = .
version = 0.1
requirements = python3,kivy
orientation = landscape

[buildozer]
log_level = 1
android.api = 29
android.minapi = 21
android.ndk = 21b
android.archs = armeabi-v7a
android.accept_sdk_license = True
"""

with open("buildozer_kivy.spec", "w") as f:
    f.write(kivy_spec)
print("✅ Created Kivy-based buildozer.spec")

# Fix 2: Ultra-minimal Python-only app
print("\\n🔧 FIX 2: Python-only approach (most reliable)")
minimal_spec = """[app]
title = UHFF Minimal
package.name = uhffmin
package.domain = com.uhff
source.dir = .
version = 0.1
requirements = python3
orientation = landscape

[buildozer]
log_level = 1
android.api = 28
android.minapi = 21
android.ndk = 20b
android.archs = armeabi-v7a
android.accept_sdk_license = True
"""

with open("buildozer_minimal.spec", "w") as f:
    f.write(minimal_spec)
print("✅ Created minimal buildozer.spec")

# Fix 3: Alternative build commands
print("\\n🔧 FIX 3: Alternative build approaches")
print("Try these commands one by one:")
print("1. !buildozer android debug --verbose")
print("2. !buildozer android update")
print("3. !buildozer android clean && buildozer android debug")

# Fix 4: Check system resources
print("\\n🔧 FIX 4: System resource check")
!df -h | grep -E "(Filesystem|/dev/root)"
!free -h

print("\\n💡 COMMON SOLUTIONS:")
solutions = [
    "Use Java 8 instead of Java 11+: !apt install openjdk-8-jdk",
    "Use older Android API: android.api = 28",
    "Use single architecture: android.archs = armeabi-v7a", 
    "Use older NDK: android.ndk = 20b",
    "Reduce requirements to just python3",
    "Increase Colab RAM if available",
    "Try different buildozer version: pip install buildozer==1.4.0"
]

for i, solution in enumerate(solutions, 1):
    print(f"  {i}. {solution}")

print("\\n🎯 RECOMMENDED NEXT STEPS:")
print("1. Copy one of the alternative specs above over buildozer.spec")
print("2. Run: !buildozer android clean")
print("3. Run: !buildozer android debug")
print("4. If still fails, try with minimal requirements first")

# Quick build test with minimal setup
print("\\n🧪 QUICK TEST: Want to try ultra-minimal build now? (y/n)")
test_now = input()
if test_now.lower() == 'y':
    !cp buildozer_minimal.spec buildozer.spec
    !buildozer android clean
    print("🚀 Starting minimal build...")
    !timeout 300 buildozer android debug || echo "Build timed out or failed"

# 🔧 Section 3: FIXING YOUR SPECIFIC ISSUES

Based on your diagnostic results, we need to address several compatibility issues:

## Issues Identified:
- ⚠️ **Python 3.12+ compatibility problems with p4a**
- ⚠️ **Java 11 vs Java 8 preference conflicts**  
- ❌ **Multi-architecture build conflicts (arm64-v8a + armeabi-v7a)**
- ❌ **NDK API level 21 configuration problems**
- ❌ **Pygame/NumPy compilation problems**

## 💡 Strategy:
1. Create optimized single-architecture buildozer.spec
2. Force compatible tool versions
3. Simplify build requirements
4. Use incremental testing approach

In [ ]:
# 🔧 STEP 1: Create Optimized Single-Architecture buildozer.spec
print("📝 Creating optimized buildozer.spec for single architecture...")

optimized_buildozer_spec = """[app]
# Basic app info
title = UHFF Visualization
package.name = uhffvisualization
package.domain = com.uhff.visualization

# Source configuration
source.dir = .
source.include_exts = py,png,jpg,kv,atlas

# Version info
version = 1.0
version.regex = __version__ = ['"](.+)['"]
version.filename = %(source.dir)s/main.py

# Requirements - MINIMAL for compatibility
requirements = python3==3.11.9,pygame==2.1.3,numpy==1.24.3

[buildozer]
# Build directory
log_level = 2
warn_on_root = 1

# SINGLE ARCHITECTURE - Key fix for your issues
android.archs = arm64-v8a

# Android configuration - COMPATIBLE VERSIONS
android.api = 31
android.minapi = 21
android.ndk = 25b
android.sdk = 31
android.gradle_dependencies = 

# Permissions
android.permissions = INTERNET,WRITE_EXTERNAL_STORAGE

# Orientation
orientation = all

# Bootstrap - Use SDL2 for pygame
android.bootstrap = sdl2

# Skip update and accept licenses
android.skip_update = False
android.accept_sdk_license = True

# Debugging
android.logcat_filters = *:S python:D

[app]
# Entry point
android.entrypoint = org.kivy.android.PythonActivity
android.activity_class_name = org.kivy.android.PythonActivity

# Java options
android.add_java_dir = 
android.gradle_dependencies = 
android.gradle_repositories = 

# Whitelist
android.whitelist = 

# Build mode
android.release_artifact = apk
android.debug_artifact = apk
"""

# Write optimized spec
with open('/content/ether/buildozer_optimized.spec', 'w') as f:
    f.write(optimized_buildozer_spec)

print("✅ Created buildozer_optimized.spec with:")
print("   - Single architecture (arm64-v8a only)")
print("   - Compatible Python 3.11.9")
print("   - Stable pygame 2.1.3")  
print("   - Numpy 1.24.3")
print("   - NDK 25b")
print("   - API 31/minAPI 21")
print("   - SDL2 bootstrap for pygame")

In [ ]:
# 🔧 STEP 2: Force Compatible Tool Versions
print("🔧 Setting up compatible build environment...")

# Force specific versions to avoid conflicts
setup_commands = [
    "pip install --upgrade pip==23.3.1",
    "pip install buildozer==1.5.0",
    "pip install cython==0.29.36", 
    "pip install setuptools==68.2.2",
    "pip install wheel==0.41.2"
]

import subprocess
import sys

for cmd in setup_commands:
    print(f"📦 Running: {cmd}")
    try:
        result = subprocess.run(cmd.split(), capture_output=True, text=True, timeout=120)
        if result.returncode == 0:
            print(f"   ✅ Success")
        else:
            print(f"   ⚠️  Warning: {result.stderr[:100]}")
    except Exception as e:
        print(f"   ❌ Error: {str(e)[:100]}")

# Set environment variables for compatibility
import os
os.environ['ANDROIDSDK'] = '/root/android-sdk'
os.environ['ANDROIDNDK'] = '/root/android-sdk/ndk/25.2.9519653'
os.environ['ANDROIDAPI'] = '31'
os.environ['NDKAPI'] = '21'

print("\n🎯 Environment configured for:")
print("   - Buildozer 1.5.0")
print("   - Cython 0.29.36 (stable)")  
print("   - Android SDK/NDK paths set")
print("   - API levels configured")

In [ ]:
# 🔧 STEP 3: Incremental Build Test (Minimal Requirements)
print("🧪 Testing with absolute minimal requirements first...")

# Create minimal test buildozer.spec
minimal_spec = """[app]
title = UHFF Test
package.name = uhfftest
package.domain = com.test.uhff
source.dir = .
source.include_exts = py
version = 1.0
requirements = python3

[buildozer]
log_level = 2
android.archs = arm64-v8a
android.api = 31
android.minapi = 21
android.ndk = 25b
android.sdk = 31
android.bootstrap = sdl2
android.accept_sdk_license = True
orientation = portrait
"""

with open('/content/ether/buildozer_minimal.spec', 'w') as f:
    f.write(minimal_spec)

print("📝 Created minimal test spec with:")
print("   - Only python3 requirement")
print("   - Single architecture")
print("   - Portrait orientation only")
print("   - SDL2 bootstrap")

# Test if basic toolchain creation works
os.chdir('/content/ether')
print("\n🚀 Testing basic toolchain creation...")
print("⏳ This will take 5-10 minutes for toolchain setup...")

# Just test toolchain creation, not full build
test_cmd = "timeout 600 buildozer android clean || echo 'Clean completed'"
print(f"Running: {test_cmd}")

In [ ]:
# 🔧 STEP 4: Alternative Python-for-Android Approach
print("🔄 Alternative approach - Direct p4a installation...")

# Sometimes buildozer's p4a fails, so let's try direct installation
alternative_setup = """
# Alternative python-for-android setup
pip uninstall -y python-for-android
pip install git+https://github.com/kivy/python-for-android.git@master

# Set up p4a directly
export ANDROIDAPI=31
export NDKAPI=21
export ANDROID_HOME=/root/android-sdk
export ANDROID_NDK_HOME=/root/android-sdk/ndk/25.2.9519653

# Create distributions with specific requirements
p4a create --arch=arm64-v8a --requirements=python3,pygame,numpy --bootstrap=sdl2 --dist-name=uhff_minimal

# If that works, then build APK
p4a apk --arch=arm64-v8a --requirements=python3,pygame,numpy --bootstrap=sdl2 --name="UHFF Test" --package=com.test.uhff --version=1.0 --private=/content/ether
"""

print("📋 Alternative commands to try if buildozer fails:")
print(alternative_setup)

# Also create a simple test main.py for minimal testing
simple_main = '''
import pygame
import sys

def main():
    pygame.init()
    screen = pygame.display.set_mode((400, 300))
    pygame.display.set_caption("UHFF Test")
    
    clock = pygame.time.Clock()
    running = True
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        screen.fill((0, 100, 200))
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()

if __name__ == "__main__":
    main()
'''

with open('/content/ether/simple_main.py', 'w') as f:
    f.write(simple_main)

print("\n✅ Created simple_main.py for testing")
print("💡 If buildozer fails, try the p4a commands above manually")

# 🚀 Section 4: RUNNING THE OPTIMIZED BUILD

## Now let's try the build with our fixes:

### Option A: Use the optimized buildozer.spec
```bash
cd /content/ether
cp buildozer_optimized.spec buildozer.spec
buildozer android clean
buildozer android debug
```

### Option B: Use minimal spec for testing  
```bash
cd /content/ether
cp buildozer_minimal.spec buildozer.spec
buildozer android clean
buildozer android debug
```

### Option C: Manual p4a approach (if buildozer fails)
Use the commands from Step 4 above.

## 🎯 What to expect:
- **Single architecture** should resolve multi-arch conflicts
- **Compatible versions** should fix NDK/API issues  
- **Minimal requirements** should reduce compilation problems
- **SDL2 bootstrap** should work better with pygame

In [ ]:
# 🎯 EXECUTE OPTIMIZED BUILD - Run this cell to start!
print("🚀 Starting optimized build process...")
print("📊 This addresses all issues from your diagnostic:")
print("   ✅ Single architecture (arm64-v8a)")
print("   ✅ Compatible Python 3.11.9")
print("   ✅ Stable pygame 2.1.3")
print("   ✅ NDK 25b + API 31")
print("   ✅ SDL2 bootstrap")

import os
import subprocess

# Change to project directory
os.chdir('/content/ether')

# Copy optimized spec
subprocess.run(['cp', 'buildozer_optimized.spec', 'buildozer.spec'])

print("\n🧹 Cleaning previous build artifacts...")
result = subprocess.run(['buildozer', 'android', 'clean'], 
                       capture_output=True, text=True, timeout=300)

if result.returncode == 0:
    print("✅ Clean completed successfully")
else:
    print(f"⚠️  Clean warning (normal): {result.stderr[:200]}")

print("\n🔨 Starting optimized debug build...")
print("⏳ This will take 15-30 minutes...")
print("💡 Watch for 'BUILD SUCCESSFUL' message")

# Start the build (this will take a while)
build_result = subprocess.run(['buildozer', 'android', 'debug'], 
                             capture_output=True, text=True, timeout=3600)

if build_result.returncode == 0:
    print("\n🎉 BUILD SUCCESSFUL! 🎉")
    print("📱 APK created: bin/uhffvisualization-1.0-debug.apk")
else:
    print(f"\n❌ Build failed: {build_result.stderr[-500:]}")
    print("\n💡 Try the alternative approaches in the next cells")

# ⚡ QUICK FIX: Build Hanging Issue

## If the previous cell is stuck at "Buildozer is running as root!" 

**The build process can hang during the toolchain setup. Here's how to fix it:**

### 🔄 **Option 1: Force Continue (Recommended)**
Run the next cell to bypass the hanging and continue with a non-interactive build.

### 🔄 **Option 2: Restart and Use Manual Commands** 
If it's completely stuck, restart the runtime and use the manual p4a commands instead.

In [ ]:
# ⚡ FORCE CONTINUE - Run this if previous cell is stuck!
print("⚡ Bypassing buildozer hanging issue...")
print("🔧 Using non-interactive build approach...")

import os
import subprocess
import signal
import time

# Kill any hanging buildozer processes
try:
    subprocess.run(['pkill', '-f', 'buildozer'], timeout=10)
    subprocess.run(['pkill', '-f', 'gradle'], timeout=10)
    print("🗑️  Killed hanging processes")
except:
    pass

# Change to project directory
os.chdir('/content/ether')

# Set environment to non-interactive
os.environ['BUILDOZER_LOG_LEVEL'] = '1'
os.environ['GRADLE_OPTS'] = '-Dorg.gradle.daemon=false'
os.environ['ANDROID_SDK_ROOT'] = '/root/android-sdk'

# Force accept all prompts with yes
print("🤖 Setting up non-interactive build...")

# Create a simple script that auto-answers 'y' to all prompts
auto_yes_script = """#!/bin/bash
echo "y" | buildozer android debug
"""

with open('/content/auto_build.sh', 'w') as f:
    f.write(auto_yes_script)

os.chmod('/content/auto_build.sh', 0o755)

print("✅ Created auto-yes build script")
print("🚀 Starting non-interactive build...")
print("⏳ This should not hang - will timeout after 30 minutes")

# Run with timeout to prevent infinite hanging
try:
    result = subprocess.run(['/content/auto_build.sh'], 
                           timeout=1800,  # 30 minutes max
                           capture_output=True, 
                           text=True,
                           cwd='/content/ether')
    
    if "BUILD SUCCESSFUL" in result.stdout:
        print("🎉 BUILD SUCCESSFUL!")
        print("📱 APK location: /content/ether/bin/")
    else:
        print("📋 Build output (last 1000 chars):")
        print(result.stdout[-1000:])
        if result.stderr:
            print("❌ Errors:")
            print(result.stderr[-500:])
            
except subprocess.TimeoutExpired:
    print("⏰ Build timed out - trying alternative approach...")
    print("💡 Run the next cell for manual p4a method")

In [ ]:
# 🔄 ALTERNATIVE: Manual P4A Build (If buildozer keeps hanging)
print("🛠️  Manual python-for-android approach...")
print("🎯 This bypasses buildozer completely")

import os
import subprocess

# Setup environment
os.environ['ANDROID_HOME'] = '/root/android-sdk'
os.environ['ANDROID_NDK_HOME'] = '/root/android-sdk/ndk/25.2.9519653'
os.environ['ANDROIDAPI'] = '31'
os.environ['NDKAPI'] = '21'

# Install p4a directly
print("📦 Installing python-for-android...")
subprocess.run(['pip', 'install', 'python-for-android==2023.6.11'], 
               capture_output=True)

print("✅ P4A installed")

# Create distribution
print("🔧 Creating p4a distribution...")
os.chdir('/content/ether')

p4a_commands = [
    # Clean any existing
    ['p4a', 'clean_all'],
    
    # Create distribution with minimal requirements
    ['p4a', 'create', '--arch=arm64-v8a', '--bootstrap=sdl2',
     '--requirements=python3,pygame', '--dist-name=uhff'],
    
    # Build APK
    ['p4a', 'apk', '--arch=arm64-v8a', '--bootstrap=sdl2',
     '--requirements=python3,pygame', '--dist-name=uhff',
     '--name=UHFF', '--package=com.uhff.app', '--version=1.0',
     '--private=.', '--orientation=portrait']
]

for i, cmd in enumerate(p4a_commands, 1):
    print(f"\n📋 Step {i}: {' '.join(cmd[:3])}...")
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=600)
        if result.returncode == 0:
            print(f"✅ Step {i} completed")
        else:
            print(f"❌ Step {i} failed: {result.stderr[:200]}")
            if i == 1:  # Clean can fail, continue anyway
                continue
            else:
                break
    except subprocess.TimeoutExpired:
        print(f"⏰ Step {i} timed out")
        break
    except Exception as e:
        print(f"❌ Step {i} error: {str(e)}")
        break

# Check for APK
apk_files = subprocess.run(['find', '.', '-name', '*.apk'], 
                          capture_output=True, text=True)
if apk_files.stdout:
    print(f"\n🎉 APK FOUND!")
    print(f"📱 Location: {apk_files.stdout.strip()}")
else:
    print("\n❌ No APK generated")
    print("💡 Try the emergency fixes in the next section")